In [16]:
"""
Initial setup:
1) create virtualenv project using pycharm

2) install the following libraries to virtualenv:
pip install numpy
pip install pandas
pip install requests
pip install beautifulsoup4
pip install lxml
"""

'\nInitial setup:\n1) create virtualenv project using pycharm\n\n2) install the following libraries to virtualenv:\npip install numpy\npip install pandas\npip install requests\npip install beautifulsoup4\npip install lxml\n'

In [19]:
import requests  # library to send requests to web site(krisha.kz)
from bs4 import BeautifulSoup as bs  # library to copy all html-code
import time
import math
import re
import json
from lxml import html
import regex
import pandas as pd

In [24]:
headers = {
    'accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
}

items_per_page = 12

def get_all_pages(catalog_page_url):
    urls = []
    urls.append(catalog_page_url)
    session = requests.Session()
    request = session.get(catalog_page_url, headers=headers)

    if request.status_code == 200:
        soup = bs(request.content, 'lxml')
        try:
            pagination = soup.select('li.tree__item._expanded._active')
            total_items = int(re.sub('\D', '', pagination[0].text))
            pages = math.ceil(total_items / items_per_page)
            for i in range(2, pages):
                url = catalog_page_url + '?page={i}'.format(i=i)
                if url not in urls:
                    urls.append(url)
        except Exception as e:
            print("exception while getting all page urls: " + e)
            pass
    return urls

def parse_kaspi_pages(urls):
    items = []
    session = requests.Session()
    c = 0
    for url in urls:
        time.sleep(1)
        print("parsing: " + url)
        request = session.get(url, headers=headers)
        tree = html.fromstring(request.content)
        script = tree.xpath('//script[contains(., "BACKEND.components.catalogGrid")]/text()')[0]
        data = regex.compile(r"\{(?:[^{}]|(?R))*\}").findall(str(script))[0]
        all_data = json.loads(data)
        cards = all_data["cards"]
        for card in cards:
            card.pop("monthlyInstallment", None)
            card.pop("previewImages", None)
            card.pop("category", None)
        items += cards
        c += 1
        if c > 2:
            break
    return items

base_url = 'https://kaspi.kz/shop/c/notebooks/'
urls = get_all_pages(base_url)
items = parse_kaspi_pages(urls)
df = pd.json_normalize(items)
df.to_csv('csvfile.csv', encoding='utf-8', index=False)

parsing: https://kaspi.kz/shop/c/notebooks/
parsing: https://kaspi.kz/shop/c/notebooks/?page=2
parsing: https://kaspi.kz/shop/c/notebooks/?page=3


In [22]:
import csv

items = []
with open('Kaspi_data.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    initial = True
    for row in spamreader:
        if not initial:
            title = row[0].split(" ")
            price = re.sub('\D', '', row[1])
            rating = row[2]

            items.append({
                'brand': title[0],
                'color': title[-1],
                'full_name': ' '.join(title[:-1]),
                'price': price,
                'rating': rating,
            })
        else:
            initial = False

print(items[0])
def files_writer(items):
    # with open('HeadHunter.csv', 'a', encoding='utf-8') as file:
    with open(r"Kaspi_data.csv", "w", encoding='utf-8') as file:
        a_pen = csv.writer(file)
        a_pen.writerow(('brand', 'color', 'full_name', 'price', 'rating'))
        for item in items:
            a_pen.writerow((item['brand'], item['color'], item['full_name'], item['price'], item['rating']))

files_writer(items)

{'brand': 'Lenovo', 'color': 'серый', 'full_name': 'Lenovo V14-ADA 82C6S03900', 'price': '184800', 'rating': '297'}
